In [ ]:
clipfolder = '/home/jasp/mystuff/dgt/imdrefgrid/'
rst        = '/home/jasp/mystuff/dgt/imd_pt.tif'

outfolder  = '/home/jasp/mystuff/dgt/imdgrid/'

In [ ]:
from glass.pys.oss import lst_ff, fprop
from glass.wenv.grs import run_grass, rst_to_region
from glass.prop.prj import get_epsg
from glass.prop.rst import rst_cellsize
from glass.dtt.torst import shp_to_rst
import os

In [ ]:
shps = lst_ff(clipfolder, file_format='.shp')

In [ ]:
rbname = fprop(rst, 'fn')

loc = f'loc_{rbname}'

gb = run_grass(outfolder, location=loc, srs=rst, grassBIN="grass78")

import grass.script.setup as gsetup

gsetup.init(gb, outfolder, loc, 'PERMANENT')

In [ ]:
from glass.it.rst import rst_to_grs, grs_to_rst, grs_to_mask

In [ ]:
# Add main raster to grass gis

mrst = rst_to_grs(rst, rbname, as_cmd=True)

In [ ]:
maincs = rst_cellsize(rst, gisApi='gdal')

In [ ]:
for shp in shps:
    rname = fprop(shp, 'fn')
    
    clprst = shp_to_rst(
        shp, None, maincs, 0, os.path.join(
            outfolder, loc,
            f"{rname}.tif"
        )
    )
    
    refgrs = rst_to_grs(clprst, rname, as_cmd=True)
    
    # Set New Region
    rst_to_region(refgrs)
    
    # Set mask
    grs_to_mask(refgrs, overwrite=True)
    
    # Export clipped raster
    cellid = rname.split('_')[-1]
    grs_to_rst(mrst, os.path.join(outfolder, f"{rbname}_{cellid}.tif"))